## Testing Julia magnetic field calculation

In [1]:
# utils 
using LinearAlgebra

function create_grid(xlim=(-1,1), ylim=(-1,1), zlim=(-1,1), n=(10,10,10))
    x = range(xlim[1], stop=xlim[2], length=n[1])
    y = range(ylim[1], stop=ylim[2], length=n[2])
    z = range(zlim[1], stop=zlim[2], length=n[3])
    grid = zeros(n[1], n[2], n[3], 3)
    for i in 1:n[1], j in 1:n[2], k in 1:n[3]
        grid[i,j,k,1] = x[i]
        grid[i,j,k,2] = y[j]
        grid[i,j,k,3] = z[k]
    end
    grid = reshape(grid, n[1]*n[2]*n[3], 3)
    return grid
end

function create_horiz_circular_path(n=8, r=2.0, z=1.0)
    # create a wire path
    t = range(0, stop=2π, length=n+1)
    x = r * cos.(t)
    y = r * sin.(t)
    z = fill(z, n+1)
    wp = [x y z]
    return wp
end;

In [2]:
# wire and magnetic field
using LinearAlgebra
using Plots; 
# plotlyjs()
using ProgressMeter
include("utils.jl")

const myT = Float64 # type of the wire: Float32 or Float64

mutable struct FemWire
    wp::Array{myT, 2}
    V::myT
    ρ::myT
    section::myT
    seg_len::myT
    R::myT
    L::myT
    I::myT
    
    # constructor with calculated parameters R L I
    function FemWire(wp::Array{myT, 2}, V::myT, ρ::myT, section::myT, seg_len::myT)
        # calculate length of the wire L, and resample the wire path with seg_len
        L = path_length(wp) # length L = path_length(wp) 
        R = ρ * L / section # resistance R = ρ * L / A
        I = V / R # current I = V / R
        # resample the wire path with seg_len
        wp = upresample(wp, seg_len) # resample the wire path
        return new(wp, V, ρ, section, seg_len, R, L, I)
    end
end

function calc_mf(ws::Array{FemWire}, fem_grid::Array{myT, 2}) 
    # calculate B field on a fem_grid
    @assert size(fem_grid, 2) == 3 "fem_grid must be a (n,3) array, not $(size(fem_grid))"
    B = zeros(size(fem_grid,1), 3) # B field
    # calculate B field, n=fem_grid points, m=wire points
    μ0 = 4 * π * 1e-7 # vacuum permeability
    for w in wires # loop over wires        
        for i in axes(fem_grid, 1) # loop over fem_grid points
            for j in axes(w.wp, 1)
                wp1, wp2 = w.wp[j,:], w.wp[j % size(w.wp,1) + 1,:] # wire points
                dl = wp2 - wp1 # dl 
                wm = (wp1 + wp2) / 2 # wire midpoint
                r = wm - fem_grid[i,:] # r 
                rnorm = norm(r) # r norm 
                r̂ = r / rnorm # unit vector r 
                B[i,:] += μ0 * w.I * cross(dl, r̂) / (4 * π * rnorm^2) # Biot-Savart law
            end
        end
    end
    return B
end;

In [23]:
# test all the functions
using Plots
plotlyjs()

const N_GRID = 30
const GRID_RANGE = (-1.5,1.5)

gr = create_grid(GRID_RANGE,GRID_RANGE,GRID_RANGE, (N_GRID, N_GRID, N_GRID))

wp1 = create_horiz_circular_path(3, 2.0, -1.0)
wp2 = create_horiz_circular_path(5, 2.0, 1.5)

# # plot wire paths
# p1 = plot() # create a plot
# plot!(wp1[:,1], wp1[:,2], wp1[:,3], label="wp1", line=(:red, 3))
# plot!(wp2[:,1], wp2[:,2], wp2[:,3], label="wp2", line=(:blue, 3))
# display(p1) # display the plot

w1 = FemWire(wp1, 150.0, 1.77e-8, 1e-4, 0.01)
w2 = FemWire(wp2, 100.0, 1.77e-8, 1e-4, 0.01) 

wires = [w1, w2]
# wires = [w1]
println("wires created")

B = calc_mf(wires, gr)

# quiver 3d plot of the magnetic field
normB = B ./ norm.(B) * 0.1

In [ ]:
p2 = plot() # create a plot
u, v, w = normB[:,1], normB[:,2], normB[:,3]
# quiver!(gr[:,1], gr[:,2], gr[:,3], quiver=(u, v, w),color=:blues, colorbar_title="B", normalize=true)
arrow3d!(gr[:,1], gr[:,2], gr[:,3], u, v, w, as=0.2, lc=:black, la=0.5, lw=2)
# plot wire paths
plot!(wp1[:,1], wp1[:,2], wp1[:,3], label="wp1", line=(:red, 3))
plot!(wp2[:,1], wp2[:,2], wp2[:,3], label="wp2", line=(:blue, 3))
display(p2) # display the plot